In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pandas as pd
import time

import random

In [52]:
DRIVER_PATH = r'Z:\РАБОЧИЕ ДОКУМЕНТЫ\1.БАРОМЕТР\СИСТЕМА  БАРОМЕТР Fine Report  2024\4. webdrivers\chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

In [54]:
# gathering the companies which start with numbers (they dont have their own category on the website)
link = 'https://www.business-standard.com/markets/stock-companies-list'
driver.get(f'''{link}''')
time.sleep(0.5)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
WebDriverWait(driver, 5, poll_frequency=random.uniform(0, 1)+0.5).until(
    EC.presence_of_element_located((
        By.CSS_SELECTOR, f'''#__next > div > div.section-div.corporate-box.gl-table.no-pad > div.quote-head.load-more'''))).click()
time.sleep(1)

# getting a list of all the links and using the last one (as it is the most relveant)
content = driver.find_elements(By.TAG_NAME, "a")
links = [elem.get_attribute('href') for elem in content]

begin = links.index( 'https://www.business-standard.com/markets/stock-companies-list/z')
end = links.index('https://www.business-standard.com/markets/7seas-entertainment-ltd-share-price-28404.html')

links_companies = links[begin+1:end+1]

india_comps = pd.DataFrame({link : links_companies}).T

In [3]:
# compiled by hand from the website
source = ['https://www.business-standard.com/markets/stock-companies-list/a',
          'https://www.business-standard.com/markets/stock-companies-list/b',          
          'https://www.business-standard.com/markets/stock-companies-list/c',
          'https://www.business-standard.com/markets/stock-companies-list/d',
          'https://www.business-standard.com/markets/stock-companies-list/e',
          'https://www.business-standard.com/markets/stock-companies-list/f',
          'https://www.business-standard.com/markets/stock-companies-list/g',
          'https://www.business-standard.com/markets/stock-companies-list/h',
          'https://www.business-standard.com/markets/stock-companies-list/i',
          'https://www.business-standard.com/markets/stock-companies-list/j',
          'https://www.business-standard.com/markets/stock-companies-list/k',
          'https://www.business-standard.com/markets/stock-companies-list/l',
          'https://www.business-standard.com/markets/stock-companies-list/m',
          'https://www.business-standard.com/markets/stock-companies-list/n',
          'https://www.business-standard.com/markets/stock-companies-list/o',
          'https://www.business-standard.com/markets/stock-companies-list/p',
          'https://www.business-standard.com/markets/stock-companies-list/q',         
          'https://www.business-standard.com/markets/stock-companies-list/r',
          'https://www.business-standard.com/markets/stock-companies-list/s',
          'https://www.business-standard.com/markets/stock-companies-list/t',
          'https://www.business-standard.com/markets/stock-companies-list/u',
          'https://www.business-standard.com/markets/stock-companies-list/v',
          'https://www.business-standard.com/markets/stock-companies-list/w',
          'https://www.business-standard.com/markets/stock-companies-list/x',
          'https://www.business-standard.com/markets/stock-companies-list/y',
          'https://www.business-standard.com/markets/stock-companies-list/z']

In [4]:
for link in source:
    driver.get(f'''{link}''')
    time.sleep(0.5)
    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 5, poll_frequency=random.uniform(0, 1)+0.5).until(
                EC.presence_of_element_located((
                    By.CSS_SELECTOR, f'''#__next > div > div.section-div.corporate-box.gl-table.no-pad > div.quote-head.load-more'''))).click()
            time.sleep(1)
        except:
            break
    # getting a list of all the links and using the last one (as it is the most relveant)
    content = driver.find_elements(By.TAG_NAME, "a")
    links = [elem.get_attribute('href') for elem in content]

    begin = links.index( 'https://www.business-standard.com/markets/stock-companies-list/z')
    end = links.index( 'https://www.business-standard.com/markets/suzlon-energy-ltd-share-price-13872.html')
    
    links_companies = links[begin+1:end]

    df = pd.DataFrame({link : links_companies}).T
    india_comps = pd.concat([india_comps, df])


KeyboardInterrupt



In [ ]:
# changing the wide table into a single column table
india_comps = india_comps.T

df = pd.DataFrame()
for column in india_comps.columns:
    df = pd.concat([df, india_comps[column]])

india_comps = df.rename(columns={0 : 'source'}).dropna()

In [ ]:
# going through each link and collecting data about company
for web_link in india_comps.source:
    #remebering the source
    website = {'source' : [web_link]}
    stock_info = pd.DataFrame(website)

    driver.get(f'''{web_link}''')
    time.sleep(random.uniform(0, 1)+0.5)
    
    # company name
    try:
        company_name = driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(7) > div:nth-child(2) > section > div.flex-70 > div:nth-child(2) > div > div.companytop_cmpnynmecol__9Vpqn > h1").text
    except:
        company_name = ''
    
    # BSE TICKER
    try:
        BSE_TICKER = driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(7) > div:nth-child(2) > section > div.flex-70 > div:nth-child(2) > div > div.companytop_cmpnysminfo__BN8Lc > p:nth-child(1)").text
    except:
        BSE_TICKER = ''
    
    # NSE TICKER
    try:
        NSE_TICKER = driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(7) > div:nth-child(2) > section > div.flex-70 > div:nth-child(2) > div > div.companytop_cmpnysminfo__BN8Lc > p:nth-child(2)").text
    except:
        NSE_TICKER = ''  
    
    # SECTOR
    try:
        SECTOR = driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(7) > div:nth-child(2) > section > div.flex-70 > div:nth-child(2) > div > div.companytop_cmpnysminfo__BN8Lc > p:nth-child(3)").text
    except:
        SECTOR = ''   
    
    # ISIN CODE
    try:
        ISIN = driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(7) > div:nth-child(2) > section > div.flex-70 > div:nth-child(2) > div > div.companytop_cmpnysminfo__BN8Lc > p:nth-child(4)").text
    except:
        ISIN = ''   
    
    # LAST BSE PRICE
    try:
        BSE_PRICE = driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(7) > div:nth-child(2) > section > div.flex-70 > div:nth-child(2) > div > div:nth-child(2) > div.companytop_cmpnynsepricecol__gWHxt > div:nth-child(1) > p").text
    except:
        BSE_PRICE = ''
    
    # LAST NSE PRICE
    try:
        NSE_PRICE = driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(7) > div:nth-child(2) > section > div.flex-70 > div:nth-child(2) > div > div:nth-child(2) > div.companytop_cmpnynsepricecol__gWHxt > div:nth-child(2) > p").text
    except:
        NSE_PRICE = ''
    
    # Market Cap
    try:
        market_cap = driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(7) > div:nth-child(2) > section > div.flex-70 > div.companytop_stcktabscol__Obs9V > div:nth-child(5) > div.companytop_marketmapcol__xzThi > div:nth-child(1) > table > tbody > tr:nth-child(1)").text
    except:
        market_cap = ''
    
    # putting it all together
    stock_info['company_name'] = company_name
    stock_info['BSE_TICKER'] = BSE_TICKER
    stock_info['NSE_TICKER'] = NSE_TICKER
    stock_info['SECTOR'] = SECTOR
    stock_info['ISIN'] = ISIN
    stock_info['BSE_PRICE'] = BSE_PRICE
    stock_info['NSE_PRICE'] = NSE_PRICE
    stock_info['market_cap'] = market_cap
    
    # attaching it to the main df
    stock_info_database = pd.concat([stock_info_database, stock_info])

In [ ]:
# storing data
stock_info_database.to_excel(r'C:\Users\klyukin\Downloads\india_companies_info_database.xlsx', index=True)